## Dataframe column addition

### Session id

In [3]:
import pandas as pd
import re, string
import time
import json

In [48]:
with open('Computed/ql.json') as json_file:
    ql_dict = json.load(json_file)
ql = pd.read_csv('Computed/ql.csv')[['AnonID','Query','QueryTime']]
print(ql)

num = 0
prev_id = 0

def sessionID(id):
    global num, prev_id
    if prev_id != int(id):
        num = num + 1
        prev_id = int(id)
        
    return num

ql['session_id'] = ql['AnonID'].apply(sessionID)
ql = ql[['AnonID','session_id','Query','QueryTime']]
# ql.to_csv('ql.csv')

Query Length

In [10]:
ql = pd.read_csv('Computed/ql.csv')[['AnonID','session_id','Query','QueryTime']]
print(ql)

def sessionID(query):
    ret = query.split(" ")
    while "" in ret:
        ret.remove("")
    return len(ret)


ql['length'] = ql['Query'].apply(sessionID)
ql = ql[['AnonID','session_id','Query','QueryTime','length']]

# ql.to_csv('ql.csv')

           AnonID  session_id                              Query  \
0             142           1              merit releas appear     
1             217           2                          lotteri     
2             217           2                          lotteri     
3             217           2                          vietnam     
4             217           2                          vietnam     
...           ...         ...                                ...   
3942349  24963973      238541  unit church christ clergi stole     
3942350  24963973      238541        nation cathedr gift store     
3942351  24965114      238542                          weather     
3942352  24965131      238543                       dictionari     
3942353  24967039      238544                     vienna opera     

                   QueryTime  length  
0        2006-04-22 23:51:18       3  
1        2006-03-01 11:58:51       1  
2        2006-03-27 14:10:38       1  
3        2006-05-22 17:43:4

1) Instead of giving suggestions by space bar, are we do give suggestions character by character so it sort of emulates google more?
1) No, while it would be really nice, it would add a layer of complexity that you should not handle at this time. Great idea for your Project 2 ;)

2) Should there be any filtering for explicit content?
2) Excellent question. I won't require it, since it would add complexity to the project. Having said that, I will make sure I do not use as a query anything that could return unwanted terms.

3) Currently I'm using a nested dictionary approach to approach suggestions that goes "word -> queries containing that word -> frequency of query" to figure out the most relevant document.
3) Remember that is not the highest on frequency, but instead you need to look at individual sessions and do a proportion over that (as per the equation given on project specifications). The frequency would be misleading, as it would not guarantee co-occurrence.

## Query Suggestion

In [16]:
import pandas as pd
import re, string
import time
import json


# with open('Computed/wiki.json') as json_file:
#     wiki_dict = json.load(json_file)
# wiki = pd.read_csv('Computed/wiki.csv')[['content','title','id']]
# print(wiki)
# start = time.time()

with open('Computed/ql_dict_trueid.json') as json_file:
    ql_dict = json.load(json_file)
ql = pd.read_csv('Computed/ql.csv')[['AnonID','session_id','Query','QueryTime','length']]
print(ql)
# end = time.time()
# print(end - start)

           AnonID  session_id                            Query  \
0             142           1              merit releas appear   
1             217           2                          lotteri   
2             217           2                          lotteri   
3             217           2                          vietnam   
4             217           2                          vietnam   
...           ...         ...                              ...   
3942349  24963973      238541  unit church christ clergi stole   
3942350  24963973      238541        nation cathedr gift store   
3942351  24965114      238542                          weather   
3942352  24965131      238543                       dictionari   
3942353  24967039      238544                     vienna opera   

                   QueryTime  length  
0        2006-04-22 23:51:18       3  
1        2006-03-01 11:58:51       1  
2        2006-03-27 14:10:38       1  
3        2006-05-22 17:43:42       1  
4        200

In [19]:
# print(wiki_dict['merit'])
# print(wiki_dict['merit'].keys())
# '940', '1167'

# Takes a word, returns all rows of dataframe that contain the word, that has a bigger length than num
def getRows(word, num):
#     print(ql_dict[str(word)].keys())
#     return ql[(ql['session_id'].astype('str').isin(list(ql_dict[str(word)].keys()))) & (ql['length'].astype('int64') > num)]
    return ql.iloc[list(ql_dict[str(word)].keys())][ql['length'].astype('int64') > num]

word = 'vietnam'
word = word.strip()


# print(len(word.split(" ")))
# print(getRows(word, len(word.split(" "))))
# print(ql[ql['length'].astype('int64') > 2])

Rows = getRows(word, len(word.split(" ")))
# print(Rows) # This prints all rows that contain the row
result = Rows[['session_id','Query','QueryTime']].groupby(["Query","session_id"]).count().rename(columns={"QueryTime":"Occurance_per_session_id"})
result = result.reset_index()
# print(result) # This prints count for a single session
result = result.groupby(by=['Query']).count()
result['Score'] = result['Occurance_per_session_id']/len(Rows.groupby(by=["session_id"]))
# # print(str(len(Count.groupby(by=["session_id"]))) + " individual sessions")
result = result.sort_values(by=["Score"], ascending=False)[['Occurance_per_session_id', 'Score']]

result

/var/folders/kb/7q1qk5ds74sdxqbgl26pm2vw0000gp/T/ipykernel_54049/3704226885.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return ql.iloc[list(ql_dict[str(word)].keys())][ql['length'].astype('int64') > num]


,Occurance_per_session_id,Score
Query,,
vietnam war,91,0.220874
vietnam wall,11,0.026699
vietnam veteran,10,0.024272
presid south vietnam,9,0.021845
vietnam memori,7,0.016990
...,...,...
pictur vietnam,1,0.002427
pictur food vietnam,1,0.002427
phu bai vietnam,1,0.002427


In [4]:
print(list(ql_dict[str('vietnam')].keys()))
vietnam: id 

['3', '4', '1137', '1138', '1139', '1140', '1276', '3176', '8214', '12992', '14727', '33053', '33054', '33055', '40881', '51372', '51373', '51374', '51375', '51376', '51377', '51378', '51379', '51380', '51381', '51382', '51383', '51384', '51385', '51386', '51387', '51388', '51389', '51390', '51391', '51392', '51393', '51394', '51395', '51396', '51397', '51398', '51399', '51400', '51401', '51402', '51403', '51404', '51405', '51406', '51407', '51408', '51409', '51410', '51411', '51412', '51413', '54726', '55203', '55204', '55205', '55206', '73690', '79834', '80139', '80461', '80462', '80463', '80464', '80465', '80467', '80481', '94452', '95715', '95719', '95722', '106768', '133626', '137748', '153065', '156896', '159219', '159220', '159226', '159227', '159228', '159229', '159230', '159236', '159237', '159238', '159239', '165241', '165242', '165249', '165250', '165251', '165252', '165253', '165254', '170867', '176418', '176419', '176423', '176424', '176425', '176426', '176427', '176428', 